In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2


train_data_dir = "C:\\Users\\SIDDI NIKHILESH\\Music\\sign app\\data"
val_data_dir = "C:\\Users\\SIDDI NIKHILESH\\Music\\sign app\\data"


datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128), 
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    val_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


class_indices = train_generator.class_indices
print("\nClass indices (label mapping):")
for class_name, index in class_indices.items():
    print(f"Class: '{class_name}' -> Index: {index}")


base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False 

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])


model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_sign_language_model.keras', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)


history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)


base_model.trainable = True


model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Save the final model
model.save('final_sign_language_model_v3.h5')


print("\nFinal Class indices (label mapping):")
for class_name, index in class_indices.items():
    print(f"Class: '{class_name}' -> Index: {index}")


Found 960 images belonging to 6 classes.
Found 240 images belonging to 6 classes.

Class indices (label mapping):
Class: 'Done' -> Index: 0
Class: 'HowareYou' -> Index: 1
Class: 'Hungry' -> Index: 2
Class: 'MakeFavour' -> Index: 3
Class: 'Notwell' -> Index: 4
Class: 'Thanks' -> Index: 5


C:\Users\SIDDI NIKHILESH\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.6109 - loss: 1.1995

C:\Users\SIDDI NIKHILESH\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 358ms/step - accuracy: 0.6154 - loss: 1.1843 - val_accuracy: 0.9542 - val_loss: 0.1509 - learning_rate: 0.0010
Epoch 2/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 294ms/step - accuracy: 0.8726 - loss: 0.3547 - val_accuracy: 0.9792 - val_loss: 0.0515 - learning_rate: 0.0010
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 282ms/step - accuracy: 0.9355 - loss: 0.2168 - val_accuracy: 0.9875 - val_loss: 0.0518 - learning_rate: 0.0010
Epoch 4/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - accuracy: 0.9576 - loss: 0.1196 - val_accuracy: 0.9750 - val_loss: 0.0657 - learning_rate: 0.0010
Epoch 5/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.9510 - loss: 0.1327
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 276ms/step - accuracy: 0.9511 - loss: 0.1327 - val_accuracy: 0.9833 - val_loss: 0.0614 - learning_rate: 0.0010
Epoch 6/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - accuracy: 0.9585 - loss: 0.1267 - val_accura


Final Class indices (label mapping):
Class: 'Done' -> Index: 0
Class: 'HowareYou' -> Index: 1
Class: 'Hungry' -> Index: 2
Class: 'MakeFavour' -> Index: 3
Class: 'Notwell' -> Index: 4
Class: 'Thanks' -> Index: 5


In [3]:
# During model training (example)
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Save the history object to a file or as a variable to use later
history_dict = history.history  # contains accuracy, loss, val_loss, etc.


NameError: name 'model' is not defined